#Importing The Libraries
    

In [41]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

#Training set and the Test set

In [42]:
training_set_1 = pd.read_csv('/u50_1.base', delimiter = ',')
training_set_1 = np.array(training_set_1, dtype = 'int')
test_set_1 = pd.read_csv('/u50_1.test', delimiter = ',')
test_set_1 = np.array(test_set_1, dtype = 'int')

training_set_2 = pd.read_csv('/u50_2.base', delimiter = ',')
training_set_2 = np.array(training_set_1, dtype = 'int')
test_set_2 = pd.read_csv('/u50_2.test', delimiter = ',')
test_set_2 = np.array(test_set_1, dtype = 'int')

training_set_3 = pd.read_csv('/u50_3.base', delimiter = ',')
training_set_3 = np.array(training_set_1, dtype = 'int')
test_set_3 = pd.read_csv('/u50_3.test', delimiter = ',')
test_set_3 = np.array(test_set_1, dtype = 'int')

training_set_4 = pd.read_csv('/u50_4.base', delimiter = ',')
training_set_4 = np.array(training_set_1, dtype = 'int')
test_set_4 = pd.read_csv('/u50_4.test', delimiter = ',')
test_set_4 = np.array(test_set_1, dtype = 'int')

training_set_5 = pd.read_csv('/u50_5.base', delimiter = ',')
training_set_5 = np.array(training_set_1, dtype = 'int')
test_set_5 = pd.read_csv('/u50_5.test', delimiter = ',')
test_set_5 = np.array(test_set_1, dtype = 'int')


# Getting the number of users and movies

In [43]:
user_count_training_set = int(max(training_set_1[:, 0]))
user_count_test_set = int(max(test_set_1[:, 0]))
user_count = int(max(user_count_test_set, user_count_training_set))

movies_count_training_set = int(max(training_set_1[:, 1]))
movies_count_test_set = int(max(test_set_1[:, 1]))
movies_count = int(max(movies_count_test_set, movies_count_training_set))


# Converting the data into an array with users in rows and movies in columns

In [44]:
def modify(data):
  listoflist = []
  for ui in range(1, user_count+1):
    mi =data[:, 1][data[:, 0]==ui]
    ri =data[:, 2][data[:, 0]==ui]
    listofratings =np.zeros(movies_count)
    listofratings[mi-1] =ri
    listoflist.append(list(listofratings))
  return listoflist


#Modifying training set

In [45]:
training_set_1 = modify(training_set_1)
test_set_1 = modify(test_set_1)

training_set_2 = modify(training_set_2)
test_set_2 = modify(test_set_2)

training_set_3 = modify(training_set_3)
test_set_3 = modify(test_set_3)

training_set_4 = modify(training_set_4)
test_set_4 = modify(test_set_4)

training_set_5 = modify(training_set_5)
test_set_5 = modify(test_set_5)

# Converting the data into Torch tensors

In [46]:

training_set_1=torch.FloatTensor(training_set_1)
test_set_1=torch.FloatTensor(test_set_1)

training_set_2=torch.FloatTensor(training_set_2)
test_set_2=torch.FloatTensor(test_set_2)

training_set_3=torch.FloatTensor(training_set_3)
test_set_3=torch.FloatTensor(test_set_3)

training_set_4=torch.FloatTensor(training_set_4)
test_set_4=torch.FloatTensor(test_set_4)

training_set_5=torch.FloatTensor(training_set_5)
test_set_5=torch.FloatTensor(test_set_5)


# Converting the data into Torch tensors

In [47]:
training_set_1=torch.FloatTensor(training_set_1)
test_set_1=torch.FloatTensor(test_set_1)

training_set_2=torch.FloatTensor(training_set_2)
test_set_2=torch.FloatTensor(test_set_2)

training_set_3=torch.FloatTensor(training_set_3)
test_set_3=torch.FloatTensor(test_set_3)

training_set_4=torch.FloatTensor(training_set_4)
test_set_4=torch.FloatTensor(test_set_4)

training_set_5=torch.FloatTensor(training_set_5)
test_set_5=torch.FloatTensor(test_set_5)

# Creating the architecture of the Neural Network

In [48]:
class MODELL(nn.Module):
    def __init__(self, ):
        super(MODELL, self).__init__()
        self.layer1 =nn.Linear(movies_count, 20)
        self.layer2 =nn.Linear(20, 10)
        self.layer3 =nn.Linear(10, 20)
        self.layer4 =nn.Linear(20, movies_count)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x =self.activation(self.layer1(x))
        x =self.activation(self.layer2(x))
        x =self.activation(self.layer3(x))
        x =self.layer4(x)
        return x

archi = MODELL()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(archi.parameters(), lr = 0.01, weight_decay = 0.5)

# Training the Autoencoder part : 1

In [49]:
e_counts = 100
for epoch in range(e_counts):
  train_loss1 , s1 =0 , 0.                                    
  for ui in range(user_count):
    input_info = Variable(training_set_1[ui]).unsqueeze(0)
    target_info = input_info.clone()
    if torch.sum(target_info.data > 0) > 0:
      output_info =archi(input_info)
      target_info.require_grad = False
      output_info[target_info ==0] =0
      l =criterion(output_info, target_info)
      adj_cons = movies_count/float(torch.sum(target_info.data > 0) + 1e-10)
      l.backward()
      train_loss1 +=np.sqrt(l.data*adj_cons)
      s1 +=1.
      optimizer.step()
  print('train_1 loss at epoch number '+ str(epoch+1)+ ' is ' +str(train_loss1/s1))
print()

train_1 loss at epoch number 1 is tensor(1.8681)
train_1 loss at epoch number 2 is tensor(1.1532)
train_1 loss at epoch number 3 is tensor(1.0781)
train_1 loss at epoch number 4 is tensor(1.0511)
train_1 loss at epoch number 5 is tensor(1.0382)
train_1 loss at epoch number 6 is tensor(1.0300)
train_1 loss at epoch number 7 is tensor(1.0243)
train_1 loss at epoch number 8 is tensor(1.0207)
train_1 loss at epoch number 9 is tensor(1.0182)
train_1 loss at epoch number 10 is tensor(1.0161)
train_1 loss at epoch number 11 is tensor(1.0147)
train_1 loss at epoch number 12 is tensor(1.0136)
train_1 loss at epoch number 13 is tensor(1.0125)
train_1 loss at epoch number 14 is tensor(1.0118)
train_1 loss at epoch number 15 is tensor(1.0114)
train_1 loss at epoch number 16 is tensor(1.0107)
train_1 loss at epoch number 17 is tensor(1.0101)
train_1 loss at epoch number 18 is tensor(1.0098)
train_1 loss at epoch number 19 is tensor(1.0092)
train_1 loss at epoch number 20 is tensor(1.0090)
train_1 l

# Testing the Autoencoder : part 1



In [50]:

test_loss1 ,s1  = 0 , 0.
 
for ui in range(user_count):
  input_info= Variable(training_set_1[ui]).unsqueeze(0)
  target_info =Variable(test_set_1[ui]).unsqueeze(0)
  if torch.sum(target_info.data>0)> 0:
    output_info =archi(input_info)
    target_info.require_grad= False
    output_info[target_info== 0] = 0
    l= criterion(output_info, target_info)
    adj_cons =movies_count/float(torch.sum(target_info.data > 0) + 1e-10)
    test_loss1 += np.sqrt(l.data*adj_cons)
    s1+= 1.
print('test_1 loss: '+str(test_loss1/s1))
print()


archi = MODELL()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(archi.parameters(), lr = 0.01, weight_decay = 0.5)


test_1 loss: tensor(0.9848)



#Training the Autoencoder : part 2

In [51]:
e_counts = 100
for epoch in range(e_counts):
  train_loss2 , s2 =0 , 0.                                    
  for ui in range(user_count):
    input_info = Variable(training_set_2[ui]).unsqueeze(0)
    target_info = input_info.clone()
    if torch.sum(target_info.data > 0) > 0:
      output_info =archi(input_info)
      target_info.require_grad = False
      output_info[target_info ==0] =0
      l =criterion(output_info, target_info)
      adj_cons = movies_count/float(torch.sum(target_info.data > 0) + 1e-10)
      l.backward()
      train_loss2 +=np.sqrt(l.data*adj_cons)
      s2 +=1.
      optimizer.step()
  print('train_2 loss at epoch number '+ str(epoch+1)+ ' is ' +str(train_loss2/s2))
print()


train_2 loss at epoch number 1 is tensor(1.8684)
train_2 loss at epoch number 2 is tensor(1.1532)
train_2 loss at epoch number 3 is tensor(1.0782)
train_2 loss at epoch number 4 is tensor(1.0511)
train_2 loss at epoch number 5 is tensor(1.0375)
train_2 loss at epoch number 6 is tensor(1.0301)
train_2 loss at epoch number 7 is tensor(1.0249)
train_2 loss at epoch number 8 is tensor(1.0205)
train_2 loss at epoch number 9 is tensor(1.0183)
train_2 loss at epoch number 10 is tensor(1.0163)
train_2 loss at epoch number 11 is tensor(1.0148)
train_2 loss at epoch number 12 is tensor(1.0138)
train_2 loss at epoch number 13 is tensor(1.0132)
train_2 loss at epoch number 14 is tensor(1.0114)
train_2 loss at epoch number 15 is tensor(1.0115)
train_2 loss at epoch number 16 is tensor(1.0103)
train_2 loss at epoch number 17 is tensor(1.0103)
train_2 loss at epoch number 18 is tensor(1.0095)
train_2 loss at epoch number 19 is tensor(1.0095)
train_2 loss at epoch number 20 is tensor(1.0092)
train_2 l

#Testing the Autoencode : part 2

In [52]:
test_loss2 ,s2  = 0 , 0.
 
for ui in range(user_count):
  input_info= Variable(training_set_2[ui]).unsqueeze(0)
  target_info =Variable(test_set_2[ui]).unsqueeze(0)
  if torch.sum(target_info.data>0)> 0:
    output_info =archi(input_info)
    target_info.require_grad= False
    output_info[target_info== 0] = 0
    l= criterion(output_info, target_info)
    adj_cons =movies_count/float(torch.sum(target_info.data > 0) + 1e-10)
    test_loss2 += np.sqrt(l.data*adj_cons)
    s2+= 1.
print('test_2 loss: '+str(test_loss2/s2))
print()


archi = MODELL()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(archi.parameters(), lr = 0.01, weight_decay = 0.5)


test_2 loss: tensor(0.9835)



#Training the autoencoder : part 3

In [53]:

e_counts = 100
for epoch in range(e_counts):
  train_loss3 , s3 =0 , 0.                                    
  for ui in range(user_count):
    input_info = Variable(training_set_3[ui]).unsqueeze(0)
    target_info = input_info.clone()
    if torch.sum(target_info.data > 0) > 0:
      output_info =archi(input_info)
      target_info.require_grad = False
      output_info[target_info ==0] =0
      l =criterion(output_info, target_info)
      adj_cons = movies_count/float(torch.sum(target_info.data > 0) + 1e-10)
      l.backward()
      train_loss3 +=np.sqrt(l.data*adj_cons)
      s3 +=1.
      optimizer.step()
  print('train_3 loss at epoch number '+ str(epoch+1)+ ' is ' +str(train_loss3/s3))
print()

train_3 loss at epoch number 1 is tensor(1.8688)
train_3 loss at epoch number 2 is tensor(1.1530)
train_3 loss at epoch number 3 is tensor(1.0780)
train_3 loss at epoch number 4 is tensor(1.0511)
train_3 loss at epoch number 5 is tensor(1.0375)
train_3 loss at epoch number 6 is tensor(1.0298)
train_3 loss at epoch number 7 is tensor(1.0247)
train_3 loss at epoch number 8 is tensor(1.0206)
train_3 loss at epoch number 9 is tensor(1.0185)
train_3 loss at epoch number 10 is tensor(1.0161)
train_3 loss at epoch number 11 is tensor(1.0145)
train_3 loss at epoch number 12 is tensor(1.0133)
train_3 loss at epoch number 13 is tensor(1.0127)
train_3 loss at epoch number 14 is tensor(1.0115)
train_3 loss at epoch number 15 is tensor(1.0112)
train_3 loss at epoch number 16 is tensor(1.0102)
train_3 loss at epoch number 17 is tensor(1.0103)
train_3 loss at epoch number 18 is tensor(1.0097)
train_3 loss at epoch number 19 is tensor(1.0092)
train_3 loss at epoch number 20 is tensor(1.0090)
train_3 l

#Testing the autoencoder : part 3

In [54]:
test_loss3 ,s3  = 0 , 0.
 
for ui in range(user_count):
  input_info= Variable(training_set_3[ui]).unsqueeze(0)
  target_info =Variable(test_set_3[ui]).unsqueeze(0)
  if torch.sum(target_info.data>0)> 0:
    output_info =archi(input_info)
    target_info.require_grad= False
    output_info[target_info== 0] = 0
    l= criterion(output_info, target_info)
    adj_cons =movies_count/float(torch.sum(target_info.data > 0) + 1e-10)
    test_loss3 += np.sqrt(l.data*adj_cons)
    s3+= 1.
print('test_3 loss: '+str(test_loss3/s3))
print()

archi = MODELL()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(archi.parameters(), lr = 0.01, weight_decay = 0.5)

test_3 loss: tensor(0.9847)



#Training the autoencoder : part 4

In [55]:
e_counts = 100
for epoch in range(e_counts):
  train_loss4 , s4 =0 , 0.                                    
  for ui in range(user_count):
    input_info = Variable(training_set_4[ui]).unsqueeze(0)
    target_info = input_info.clone()
    if torch.sum(target_info.data > 0) > 0:
      output_info =archi(input_info)
      target_info.require_grad = False
      output_info[target_info ==0] =0
      l =criterion(output_info, target_info)
      adj_cons = movies_count/float(torch.sum(target_info.data > 0) + 1e-10)
      l.backward()
      train_loss4 +=np.sqrt(l.data*adj_cons)
      s4 +=1.
      optimizer.step()
  print('train_4 loss at epoch number '+ str(epoch+1)+ ' is ' +str(train_loss4/s4))
print()

train_4 loss at epoch number 1 is tensor(1.8683)
train_4 loss at epoch number 2 is tensor(1.1530)
train_4 loss at epoch number 3 is tensor(1.0781)
train_4 loss at epoch number 4 is tensor(1.0515)
train_4 loss at epoch number 5 is tensor(1.0378)
train_4 loss at epoch number 6 is tensor(1.0297)
train_4 loss at epoch number 7 is tensor(1.0247)
train_4 loss at epoch number 8 is tensor(1.0213)
train_4 loss at epoch number 9 is tensor(1.0182)
train_4 loss at epoch number 10 is tensor(1.0163)
train_4 loss at epoch number 11 is tensor(1.0148)
train_4 loss at epoch number 12 is tensor(1.0134)
train_4 loss at epoch number 13 is tensor(1.0128)
train_4 loss at epoch number 14 is tensor(1.0118)
train_4 loss at epoch number 15 is tensor(1.0111)
train_4 loss at epoch number 16 is tensor(1.0104)
train_4 loss at epoch number 17 is tensor(1.0099)
train_4 loss at epoch number 18 is tensor(1.0100)
train_4 loss at epoch number 19 is tensor(1.0096)
train_4 loss at epoch number 20 is tensor(1.0090)
train_4 l

#Testing the autoencoder : part 4

In [56]:

test_loss4 ,s4  = 0 , 0.
 
for ui in range(user_count):
  input_info= Variable(training_set_4[ui]).unsqueeze(0)
  target_info =Variable(test_set_4[ui]).unsqueeze(0)
  if torch.sum(target_info.data>0)> 0:
    output_info =archi(input_info)
    target_info.require_grad= False
    output_info[target_info== 0] = 0
    l= criterion(output_info, target_info)
    adj_cons =movies_count/float(torch.sum(target_info.data > 0) + 1e-10)
    test_loss4 += np.sqrt(l.data*adj_cons)
    s4+= 1.
print('test_4 loss: '+str(test_loss4/s4))
print()

archi = MODELL()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(archi.parameters(), lr = 0.01, weight_decay = 0.5)


test_4 loss: tensor(0.9858)



#Training the autoencoder : part 5

In [57]:
e_counts = 100
for epoch in range(e_counts):
  train_loss5 , s5 =0 , 0.                                    
  for ui in range(user_count):
    input_info = Variable(training_set_5[ui]).unsqueeze(0)
    target_info = input_info.clone()
    if torch.sum(target_info.data > 0) > 0:
      output_info =archi(input_info)
      target_info.require_grad = False
      output_info[target_info ==0] =0
      l =criterion(output_info, target_info)
      adj_cons = movies_count/float(torch.sum(target_info.data > 0) + 1e-10)
      l.backward()
      train_loss5 +=np.sqrt(l.data*adj_cons)
      s5 +=1.
      optimizer.step()
  print('train_5 loss at epoch number '+ str(epoch+1)+ ' is ' +str(train_loss5/s5))
print()


train_5 loss at epoch number 1 is tensor(1.8679)
train_5 loss at epoch number 2 is tensor(1.1530)
train_5 loss at epoch number 3 is tensor(1.0779)
train_5 loss at epoch number 4 is tensor(1.0510)
train_5 loss at epoch number 5 is tensor(1.0377)
train_5 loss at epoch number 6 is tensor(1.0298)
train_5 loss at epoch number 7 is tensor(1.0249)
train_5 loss at epoch number 8 is tensor(1.0209)
train_5 loss at epoch number 9 is tensor(1.0184)
train_5 loss at epoch number 10 is tensor(1.0162)
train_5 loss at epoch number 11 is tensor(1.0146)
train_5 loss at epoch number 12 is tensor(1.0134)
train_5 loss at epoch number 13 is tensor(1.0126)
train_5 loss at epoch number 14 is tensor(1.0117)
train_5 loss at epoch number 15 is tensor(1.0112)
train_5 loss at epoch number 16 is tensor(1.0106)
train_5 loss at epoch number 17 is tensor(1.0101)
train_5 loss at epoch number 18 is tensor(1.0096)
train_5 loss at epoch number 19 is tensor(1.0094)
train_5 loss at epoch number 20 is tensor(1.0090)
train_5 l

#Testing the autoencoder : part 5

In [58]:

test_loss5 ,s5  = 0 , 0.
 
for ui in range(user_count):
  input_info = Variable(training_set_5[ui]).unsqueeze(0)
  target_info = Variable(test_set_5[ui]).unsqueeze(0)
  if torch.sum(target_info.data>0)> 0:
    output_info =archi(input_info)
    target_info.require_grad= False
    output_info[target_info== 0] = 0
    l= criterion(output_info, target_info)
    adj_cons =movies_count/float(torch.sum(target_info.data > 0) + 1e-10)
    test_loss5 += np.sqrt(l.data*adj_cons)
    s5+= 1.
print('test_5 loss: '+str(test_loss5/s5))
print()


test_5 loss: tensor(0.9841)



#Calculating total error

In [59]:
total_error = ((test_loss1)/s1) + ((test_loss2)/s2) + ((test_loss3)/s3) + ((test_loss4)/s4) + ((test_loss5)/s5)
total_error /= 5.0
print("Total average Loss after 5-fold cross Validation : " + str(total_error))

Total average Loss after 5-fold cross Validation : tensor(0.9846)
